In [4]:
# import sentiment_mod as s
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cython
import nltk
import numpy as np
import random
from lemminflect import getLemma
from nltk.classify.scikitlearn import SklearnClassifier
from nltk.classify import ClassifierI
import pickle
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import spacy
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
import re
import string
nlp = spacy.load('en_core_web_lg')
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score,recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,precision_score,recall_score,confusion_matrix
from nltk.classify import ClassifierI
from statistics import mode
from sklearn.model_selection import train_test_split
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize,sent_tokenize
from xgboost import XGBClassifier
%autosave 60
# Autosaving every 60 seconds

Autosaving every 60 seconds


In [409]:
df = pd.read_csv('./data/stock_data.csv')
df

,Text,Sentiment
0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,1
1,user: AAP MOVIE. 55% return for the FEA/GEED i...,1
2,user I'd be afraid to short AMZN - they are lo...,1
3,MNTA Over 12.00,1
4,OI Over 21.37,1
...,...,...
5786,Industry body CII said #discoms are likely to ...,-1
5787,"#Gold prices slip below Rs 46,000 as #investor...",-1
5788,Workers at Bajaj Auto have agreed to a 10% wag...,1
5789,"#Sharemarket LIVE: Sensex off day’s high, up 6...",1


In [287]:
df_1 = pd.read_csv('./data/stock_tweet_data.csv',sep = ';')
df_1.dropna(inplace=True)
def create_label(text):
    if(text=='positive'):
        return 1
    else:
        return -1
df_1['sentiment'] = df_1['sentiment'].apply(lambda x:create_label(x))
df_1 = df_1[df_1['sentiment']==-1]
df_1.drop(['id','created_at'],axis=1,inplace=True)
df_1.columns = ['Text','Sentiment']
df = pd.concat([df_1,df])

In [217]:
df['label'] = df['Sentiment'].apply(lambda x: 'pos' if (x==1) else 'neg')

In [218]:
df['label'].value_counts()

pos    3685
neg    2878
Name: label, dtype: int64

In [5]:
def cleaner(text):
    text = remove_emojis(text)
    text = remove_html(text)
    text = remove_numbers(text)
    text = remove_lemma(text)
    text = text.lower()
    text = remove_pos(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = [word for word in text.split(' ') if word not in sw_nltk]
    return " ".join(words)
    
def remove_numbers(string):
    return ''.join([i for i in string if not i.isdigit()])
    
    
    
def remove_pos(text):
    result_str = ''
    doc = nlp(text)
    lemmas = []
    allowed_pos = ['ADJ','ADV','NOUN','VERB','ADP']
    for token in doc:
        if token.pos_ in allowed_pos:
            result_str +=str(token) + ' '
    return result_str

def remove_lemma(text):
    doc = nlp(text)
    lemma = []
    result_str = ''
    for token in doc:
        if token.lemma_ not in lemma:
            lemma.append(token.lemma_)
            result_str +=str(token.lemma_)+' '
    return result_str

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
    
def remove_html(text):
    clean = re.compile(r'<.*?>')
    return re.sub(clean,'',text)


def final_processing(text):
    while "  " in text:
        text = text.replace("  ", " ")
#     words = word_tokenize(text)

#     string = ''
#     for word in words:
#         lemma = lemmatizer.lemmatize(str(word),'a')
#         string+=lemma+' '

#     unique = set()
#     for word in string.split():
#         unique.add(word)
    return text

In [6]:
df['Text'] = df['Text'].apply(lambda x: cleaner(x))

In [7]:
df['Text'] = df['Text'].apply(lambda x: final_processing(x))

In [8]:
df = df[df['Text']!='']

In [9]:
all_words = []
words = df['Text'].apply(lambda x:word_tokenize(x))

In [10]:
for sent in words:
    for word in sent:
        all_words.append(word)

In [11]:
len(all_words)

49145

In [12]:
all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())
word_features = word_features[:7000]


In [13]:
document = []
pos_doc = df[df['label']=='pos']['Text']
neg_doc = df[df['label']=='neg']['Text']

In [14]:
len(neg_doc)

2102

In [15]:
for doc in pos_doc:
    document.append((doc,'pos'))
for doc in neg_doc:
    document.append((doc,'neg'))


In [16]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in words:
        features[w] = (w in word_features)
    return features

In [17]:
random.shuffle(document)
random.shuffle(all_words)

In [18]:
feature_set = [(find_features(txt),cat) for (txt,cat) in document]


In [19]:
len(feature_set)

5780

In [20]:
random.shuffle(feature_set)

In [21]:
training_set = feature_set[:5200]
testing_set = feature_set[5200:]

In [22]:
MNB_classifier = SklearnClassifier(MultinomialNB())
BNB_classifier = SklearnClassifier(BernoulliNB())
LogReg_classifier = SklearnClassifier(LogisticRegression())
KNN_classifier = SklearnClassifier(KNeighborsClassifier())
SVM_classifier = SklearnClassifier(SVC())
SGD_classifier = SklearnClassifier(SGDClassifier())
RF_classifier = SklearnClassifier(RandomForestClassifier())
XG_Classifier = SklearnClassifier(XGBClassifier())
DT_Classifier = SklearnClassifier(DecisionTreeClassifier())

In [23]:
estimators = {MNB_classifier:'Multinomial_NB_classifier',
              BNB_classifier:'Bernoulli_NB_classifier',
              LogReg_classifier:'Log_reg_classifier',
              SVM_classifier:'Support_VM_classifier',
              SGD_classifier:'Stochastic_GD_classifier',
              RF_classifier:'RandomForestClassifier',
             XG_Classifier:'XGBoost',
             DT_Classifier:'DecisionTree'}


class VoteClassifier(ClassifierI):
    
    
    def __init__(self,*classifiers):
#         self._classifiers = classifiers
        for classifier in classifiers:
            classifier.train(training_set) 
            print("%s accuracy percent:"%classifier, (nltk.classify.accuracy(classifier, testing_set))*100)
#             print('\n')
            self._classifiers = classifiers
    
    def classify(self, features):
        votes = []
        for classifier in self._classifiers:
            votes.append(classifier.classify(features))
        return mode(votes)

    def confidence(self,features):
        votes = []
        for classifier in self._classifiers:
            votes.append(classifier.classify(features))
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

    
    
voted_classifier = VoteClassifier(*estimators.keys())
true_labels = [label for (features,label) in testing_set]
pred_labels = [voted_classifier.classify(feature) for (feature,label) in testing_set]
print(classification_report(true_labels,pred_labels))
cm = confusion_matrix(true_labels,pred_labels)
print(cm)

<SklearnClassifier(MultinomialNB())> accuracy percent: 75.51724137931033
<SklearnClassifier(BernoulliNB())> accuracy percent: 75.0
<SklearnClassifier(LogisticRegression())> accuracy percent: 77.93103448275862
<SklearnClassifier(SVC())> accuracy percent: 78.96551724137932
<SklearnClassifier(SGDClassifier())> accuracy percent: 76.20689655172413
<SklearnClassifier(RandomForestClassifier())> accuracy percent: 77.24137931034483
<SklearnClassifier(XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
  

In [28]:
def sentiment(text):
    features = find_features(text)
    return (voted_classifier.classify(features),voted_classifier.confidence(features))

txt = str(input())
print(sentiment(txt))

reliance goes down 3000 points
('pos', 1.0)


In [868]:
true_labels = [label for (features,label) in testing_set]
pred_labels = [voted_classifier.classify(feature) for (feature,label) in testing_set]
print(classification_report(true_labels,pred_labels))
cm = confusion_matrix(true_labels,pred_labels)
print(cm)

KeyboardInterrupt: 

## TF-IDF

In [1150]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline,Pipeline
# cv = CountVectorizer()
tfidf = TfidfVectorizer()

In [1151]:
def cleaner(text):
    text = remove_emojis(text)
    text = remove_html(text)
    text = remove_numbers(text)
    text = remove_lemma(text)
    text = text.lower()
    text = remove_pos(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = [word for word in text.split(' ') if word not in sw_nltk]
    return " ".join(words)
    
def remove_numbers(string):
    return ''.join([i for i in string if not i.isdigit()])
    

def remove_pos(text):
    result_str = ''
    doc = nlp(text)
    allowed_pos = ['ADJ','ADV','VERB','ADP','NOUN']
    for token in doc:
        if token.pos_ in allowed_pos:
            result_str +=str(token) + ' '
    return result_str

def remove_lemma(text):
    doc = nlp(text)
    lemma = []
    result_str = ''
    for token in doc:
        if token.lemma_ not in lemma:
            lemma.append(token.lemma_)
            result_str +=str(token.lemma_)+' '
    return result_str

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)
    
def remove_html(text):
    clean = re.compile(r'<.*?>')
    return re.sub(clean,'',text)


def final_processing(text):
    while "  " in text:
        text = text.replace("  ", " ")
#     words = word_tokenize(text)

#     string = ''
#     for word in words:
#         lemma = lemmatizer.lemmatize(str(word),'a')
#         string+=lemma+' '

#     unique = set()
#     for word in string.split():
#         unique.add(word)
    return text
def remove_blank(text):
    words = word_tokenize(text)
    if len(words) < 1:
        return pd.NA
    else:
        return text
df = pd.read_csv('./data/stock_data.csv')
df_1 = pd.read_csv('./data/stock_tweet_data.csv',sep = ';')
df_1.dropna(inplace=True)

In [ ]:
def create_label(text):
    if(text=='positive'):
        return 1
    else:
        return -1
df_1['sentiment'] = df_1['sentiment'].apply(lambda x:create_label(x))
df_1 = df_1[df_1['sentiment']==-1]
df_1.drop(['id','created_at'],axis=1,inplace=True)
df_1.columns = ['Text','Sentiment']
df = pd.concat([df_1,df])
df = df.sample(frac=1)
df['Text'] = df['Text'].apply(lambda x:cleaner(x))
df['Text'] = df['Text'].apply(lambda x:final_processing(x))
df['Text'] = df['Text'].apply(lambda x:remove_blank(x))
df.dropna(inplace=True)
df = df[df['Text']!='']
df['label'] = df['Sentiment'].apply(lambda x: 'pos' if (x==1) else 'neg')

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
df['Text'] = df['Text'].apply(lambda x: re.sub(r'\bong\b', 'long', x))

In [ ]:
df['Sentiment'].value_counts()

In [ ]:
df = df.sample(frac=1)

In [ ]:
X = df['Text']
y = []
for sent in df['label']:
    if sent=='pos':
        y.append(1)
    else:
        y.append(0)
# X = cv.fit_transform(X)
X = tfidf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state = 42)

In [ ]:
# device = 'cpu'
import optuna
from sklearn.model_selection import cross_validate
tree_method = 'gpu_hist'
def RandomForest(trial):
    n_estimators = trial.suggest_int('n_estimators',100,200)
#     max_depth = trial.suggest_int('max_depth',1,10)
    clf = RandomForestClassifier(n_estimators=n_estimators)
    scores = cross_validate(clf,X,y,cv = 3,scoring=['roc_auc'])
    return (scores['test_roc_auc'].mean())
def XGB(trial):
    n_estimators = trial.suggest_int('n_estimators',100,400)
    clf = XGBClassifier(n_estimators=n_estimators,
                            eval_metric='auc',
                            objective='binary:logistic')
    scores = cross_validate(clf,X,y,cv = 3,scoring=['roc_auc'])
    return (scores['test_roc_auc'].mean())
def SVM(trial):
    C = trial.suggest_int('C',1,10)
    clf = SVC(C = C,
             kernel = 'rbf')
    scores = cross_validate(clf,X,y,cv = 3,scoring = ['roc_auc'])
    return (scores['test_roc_auc'].mean())
# def DecisionTree(trial):
#     max_depth = trial.suggest_int('max_depth',1,9)
#     clf = DecisionTreeClassifier(max_depth = max_depth,criterion = 'entropy')
#     scores = cross_validate(clf,X,y,cv = 3,scoring = ['roc_auc'])
#     return (scores['test_roc_auc'].mean())
# def NB(trial):
#     alpha = trial.suggest_int('alpha',1,10)
#     clf = BernoulliNB(alpha = alpha)
#     scores = cross_validate(clf,X,y,cv = 3,scoring=['roc_auc'])
#     return (scores['test_roc_auc'].mean())
# def KNN(trial):
#     n_neighbors = trial.suggest_int('n_neighbors',5,50)
#     clf = KNeighborsClassifier(n_neighbors = n_neighbors)
#     scores = cross_validate(clf,X,y,cv = 3,scoring=['roc_auc'])
#     return (scores['test_roc_auc'].mean())    

def Logreg(trial):
    C = trial.suggest_int('C',1,100)
    clf = LogisticRegression(C=C,max_iter=1000000)
    scores = cross_validate(clf,X,y,cv = 3,scoring=['roc_auc'])
    return (scores['test_roc_auc'].mean())


objectives_list = [SVM,XGB,Logreg,RandomForest]




scores = []
def tune(objective,est_name):
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=10)
        scores.append({'model':est_name,'best_params':study.best_params,'roc_auc':study.best_value})
        return scores
    
ensemble = [SVC,XGBClassifier,LogisticRegression,RandomForestClassifier]
for objective,estimator_name in zip(objectives_list,ensemble):
    tune(objective,estimator_name)
    
    
df_models = pd.DataFrame(scores)
df_models.sort_values(by='roc_auc',ascending=False,ignore_index=True,inplace=True)

params = df_models['best_params']

df_models

In [ ]:
df['label'].value_counts()

In [ ]:
models = []
for model,params in zip(df_models['model'],df_models['best_params']):
    models.append(model(**params))
for model in models:
    model.fit(X_train,y_train)
    
    true_labels = y_test
    pred_labels = model.predict(X_test)
    print(model)
    print('-------------------CLASSIFICATION REPORT--------------------')
    print('accuracy = ',model.score(X_test,y_test))
    print(classification_report(true_labels,pred_labels))
    cm = confusion_matrix(true_labels,pred_labels)
    print(cm)
    print('\n')

In [ ]:
from sklearn.ensemble import VotingClassifier
estimators= []
count = 1
for model in models:
    estimators.append((str(count),model))
    count+=1
# estimators

model = VotingClassifier(estimators,voting='hard')

model.fit(X_train,y_train)  
true_labels = y_test
pred_labels = model.predict(X_test)
print('-------------------CLASSIFICATION REPORT FOR ENSEMBLE--------------------')
print('accuracy = ',model.score(X_test,y_test))
print(classification_report(true_labels,pred_labels))
cm = confusion_matrix(true_labels,pred_labels)
print(cm)
print('\n')

In [ ]:
pipeline = make_pipeline(tfidf,model)

def sentiment_1(text):
    text = cleaner(text)
    text = final_processing(text)
    text_list = [text]
    sentiment = pipeline.predict(text_list)
    if sentiment==1:
        return 1
    else:
        return -1

txt = input()
print(sentiment_1(txt))

In [ ]:
import joblib
pipeline_f = open('./Pickled_algos_tfidf/pipeline.pkl','wb')
joblib.dump(pipeline,pipeline_f)
pipeline_f.close()

In [ ]:
def sentiment_1(text):
    text = cleaner(text)
    text = final_processing(text)
    text_list = [text]
    sentiment = pipeline_1.predict(text_list)
    if sentiment==1:
        return 1
    else:
        return -1

txt = input()
print(sentiment_1(txt))

In [ ]:
model = LogisticRegression(C=1)
model.fit(X_train,y_train)  
true_labels = y_test
pred_labels = model.predict(X_test)
print('-------------------CLASSIFICATION REPORT FOR %s--------------------'%str(model))
print('accuracy = ',model.score(X_test,y_test))
print(classification_report(true_labels,pred_labels))
cm = confusion_matrix(true_labels,pred_labels)
print(cm)

In [ ]:
import numpy as np
basicwords = tfidf.get_feature_names_out()
basiccoeffs = model.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(by='Coefficient',ascending=False)
coeffdf.head(20)

In [ ]:
coeffdf[coeffdf['Coefficient']<0].sum()

In [ ]:
coeffdf[coeffdf['Coefficient']>0].sum()

In [ ]:
allowed_pos = ['ADJ','ADV','VERB','INTJ','NOUN','ADP']
text = nlp('Adani Ports signs brilliant agreement to with SPMK to enhance Haldia Dock’s volume')
for token in text:
    if token.pos_ in allowed_pos:
        print(str(token) + ':' + token.pos_)